In [ ]:
!pip install torchtext==0.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 1.6 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1


In [ ]:
import torchtext.vocab as vocab

# glove = vocab.GloVe(name='840B', dim=300)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
glove = vocab.GloVe(cache="drive/MyDrive/midproject582/glove.840B.300d.zip")

In [ ]:
glove.vectors[glove.stoi["PAD"]]

In [ ]:
data_path="train.txt"
with open(data_path, 'r') as f:
    data = f.read().strip().split('\n\n')

sentences = []
for sentence in data:
    tokens = sentence.strip().split('\n')
    words = [token.split()[0] for token in tokens]
    pos_tags = [token.split()[1] for token in tokens]
    sentences.append((words, pos_tags))

In [ ]:
import numpy as np
import torch

In [ ]:

# Define the window size
window_size = 10

# Define lists to store the input features and corresponding POS tags

def get_embedding(word):
    if word in glove.stoi:
        return glove.vectors[glove.stoi[word]]
    else:
        return torch.randn(glove.dim)

def get_feature_and_pos(sentences):
    input_features = []
    pos_tags = []
    # Iterate over the dataset
    for sentence in sentences:

        # Get the list of tokens and POS tags for the sentence
        tokens = sentence[0]
        tags = sentence[1]

        # Iterate over the tokens in the sentence
        for i in range(len(tokens)):

            # Get the embeddings for the target word and its neighbors
            target_word_emb = get_embedding(tokens[i])
            neighbor_word_embs = []

            #for j in range(max(0, i-window_size), min(len(tokens), i+window_size+1)):
            for j in range(i-window_size, i+window_size+1):
                if j<0:
                    neighbor_word_emb = get_embedding("<START>")
                    neighbor_word_embs.append(neighbor_word_emb)
                elif j>=len(tokens):
                    neighbor_word_emb = get_embedding("<END>")
                    neighbor_word_embs.append(neighbor_word_emb)
                elif j != i:
                    neighbor_word_emb = get_embedding(tokens[j])
                    neighbor_word_embs.append(neighbor_word_emb)

            # Concatenate the embeddings into a single feature vector
            # print(tokens)
            feature_vector = np.concatenate((target_word_emb.numpy(), np.concatenate(neighbor_word_embs, axis=0)))

            # Add the feature vector and corresponding POS tag to the lists
            input_features.append(feature_vector)
            pos_tags.append(tags[i])

    return input_features, pos_tags


# pos_tags = torch.tensor(pos_tags, dtype=torch.float32)


In [ ]:
input_features, pos_tags = get_feature_and_pos(sentences)

# Convert the lists to numpy arrays
input_features = torch.tensor(np.array(input_features), dtype=torch.float32)

In [ ]:
# pos_vocab = np.unique(pos_tags)

pos_to_idx = {pos: i for i, pos in enumerate(pos_vocab)}

In [ ]:
pos_vocab = np.array(['#', '$', "''", '(', ')', ',', '.', ':', 'CC', 'CD', 'DT', 'EX',
       'FW', 'IN', 'JJ', 'JJR', 'JJS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS',
       'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO',
       'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$',
       'WRB', '``'], dtype='<U4')

In [ ]:
from torch.utils.data import Dataset


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MyDataset(Dataset):
    def __init__(self, input_features, pos_tags):
        self.input_features = input_features.to(device)
        self.pos_tags = torch.tensor([pos_to_idx[tag] for tag in pos_tags], dtype=torch.int8).to(device)

    def __len__(self):
        return len(self.input_features)

    def __getitem__(self, idx):
        input_feature = self.input_features[idx]
        pos_tag = self.pos_tags[idx]
        # perform any necessary data transformations here
        return input_feature, pos_tag


In [ ]:
data = MyDataset(input_features, pos_tags)

NameError: ignored

In [ ]:
del input_features

In [ ]:
torch.save(data, "drive/MyDrive/midproject582/data.pt")
# torch.save(data, "data.pt")

In [ ]:
!ls drive/MyDrive/midproject582

'data (1).pt'   data.pt   glove.840B.300d.zip


In [ ]:
data = torch.load("drive/MyDrive/midproject582/data (1).pt")

In [ ]:
data.input_features= data.input_features.to(device)
data.pos_tags=data.pos_tags.to(device)

In [ ]:
train_size = int(0.8 * len(data))
val_size = len(data) - train_size

# Split the dataset into training and validation sets
train_dataset, val_dataset = torch.utils.data.random_split(data, [train_size, val_size])

In [ ]:
input_features.shape

(211727, 3300)

In [ ]:
from torch.utils.data import DataLoader

trainloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=True)

In [ ]:
pos_vocab = np.unique(data.pos_tags.cpu().numpy())
len(pos_vocab)

44

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, output_dim)
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x


# 4. Load dataset using torch dataloader and train the model

model = MLP(6300, 44).to(device)

In [ ]:
import torch.nn.functional as F
from tqdm import tqdm

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    # mask = (y > 8) & (y < len(pos_vocab)-1)  # Create a boolean mask for the desired range of y values
    # preds = preds[mask]
    # y = y[mask]
    _, preds = torch.max(preds, dim=1)
    correct = (preds == y).float()
    acc = correct.sum() / len(correct)
    return acc


def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for i, batch in enumerate(iterator):
        optimizer.zero_grad()
        x, y = batch
        predictions = model(x)
        loss = criterion(predictions, F.one_hot(y.long(), len(pos_vocab)).float())
        acc = categorical_accuracy(predictions, y)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():
        for batch in iterator:
            x, y = batch
            predictions = model(x)
            loss = criterion(predictions, F.one_hot(y.long(), len(pos_vocab)).float())
            acc = categorical_accuracy(predictions, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
for e in tqdm(range(100)):
    loss, acc = train(model, trainloader, optimizer, criterion)
    print(f'\nloss: {loss}, acc: {acc}')
    loss, acc = evaluate(model, val_loader, criterion)
    print(f'loss: {loss}, acc: {acc}')

  0%|          | 0/100 [00:00<?, ?it/s]


loss: 3.408109373916672, acc: 0.4155518315349821


  1%|          | 1/100 [00:04<07:42,  4.67s/it]

loss: 3.372780215309327, acc: 0.44938169545437917

loss: 3.3600861936177373, acc: 0.4620318793638235


  2%|▏         | 2/100 [00:08<06:28,  3.97s/it]

...

loss: 2.9591526870267937, acc: 0.8633505131106779

loss: 2.9212599510152533, acc: 0.9012307744371927


 99%|█████████▉| 99/100 [06:06<00:03,  3.59s/it]

loss: 2.9608475702354706, acc: 0.8616660036236407

loss: 2.92291919882564, acc: 0.8995596662028679


100%|██████████| 100/100 [06:10<00:00,  3.71s/it]

loss: 2.9627684354782104, acc: 0.8596951189529465


In [ ]:
torch.save(model, "drive/MyDrive/model_window_size_10.pt")